# Global Variables

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

exchange = 'NSE'
symbol = "HDFC"
today = '2019-04-05'#dt.datetime.now().strftime("%d_%m_%y")
keyToday = '2019_04_05'
prevday = '2019-04-04'
sample_freq = '5min'

mode = "Live" # Backtesting/Live
offline = False

KiteAPIKey = "b2w0sfnr1zr92nxm"
KiteAPISecret = "jtga2mp2e5fn29h8w0pe2kb722g3dh1q"

from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 0

   

# Kite - OAuth Login

In [2]:
from kiteconnect import KiteConnect
import platform
from selenium import webdriver
import re

#logging.basicConfig(level=logging.DEBUG)

if mode == "Live":
    kite = KiteConnect(api_key=KiteAPIKey)
    #kite.login_url()

    if platform.system() == "Windows":
            driver = webdriver.Chrome("./automation/chromedriver.exe")
    else:
        driver = webdriver.Chrome("./automation/chromedriver")

    driver.get(kite.login_url())

In [3]:
if mode == "Live":
    # Redirect the user to the login url obtained
    # from kite.login_url(), and receive the request_token
    # from the registered redirect url after the login flow.
    # Once you have the request_token, obtain the access_token
    # as follows.
    request_token = re.findall("request_token=(.*)&action",driver.current_url)

    data = kite.generate_session(request_token[0], api_secret=KiteAPISecret)
    kite.set_access_token(data["access_token"])
    print(data["access_token"])
    driver.close()

zvyL9tcnN4ANGJmP3tqR7ty5AFaFynPk


# Download NSE Stock Master List

In [4]:
import datetime as dt
def getInstruments(exchange='NSE'):
    instruments_df = pd.DataFrame(data=kite.instruments(exchange))
    instruments_df = instruments_df.set_index('tradingsymbol')
    return instruments_df

def downloadData(symbol="HDFC", fromDate= dt.datetime.now() - dt.timedelta(days = 1), toDate=dt.datetime.now(), freq="minute"):
    symbolToken = instruments_df.loc[symbol,'instrument_token']
    raw_data = pd.DataFrame(data=kite.historical_data(symbolToken, fromDate, toDate, freq, continuous=False))
    raw_data = raw_data.set_index('date').tz_localize(None)
    return raw_data

def resample(data,freq):
    data = data.resample(freq).agg({'open':'first','high':'max','low':'min','close':'last','volume':'sum'})
    #data.columns = data.columns.droplevel()
    return data

In [5]:
if offline != True:
    instruments_df = getInstruments(exchange)
    instruments_df.to_hdf('kite_data/kite_data.h5', key=exchange+keyToday,mode='w')

# Download Historical Data - Equity

In [6]:
if offline != True:
    raw_data = downloadData(symbol, dt.datetime.now() - dt.timedelta(days = 3), dt.datetime.now() - dt.timedelta(days = 0))
    raw_data_day = downloadData(symbol, dt.datetime.now() - dt.timedelta(days = 2*365),dt.datetime.now(), freq="day")


    raw_data_day.to_hdf('kite_data/kite_data.h5', key=symbol+keyToday+"day",mode='a')
    raw_data.to_hdf('kite_data/kite_data.h5', key=symbol+keyToday,mode='a')

# Load Data from disk

In [7]:
raw_data = pd.read_hdf('kite_data/kite_data.h5', key=symbol+keyToday,mode='r')
raw_data_day = pd.read_hdf('kite_data/kite_data.h5', key=symbol+keyToday+"day",mode='r')

instruments_df = pd.read_hdf('kite_data/kite_data.h5', key=exchange+keyToday,mode='r')

In [8]:
raw_data.tail()

,close,high,low,open,volume
date,,,,,
2019-04-08 09:55:00,2053.90,2054.30,2052.35,2052.60,4725
2019-04-08 09:56:00,2053.65,2054.00,2051.75,2053.90,3285
2019-04-08 09:57:00,2053.95,2054.70,2052.85,2053.65,4194
2019-04-08 09:58:00,2051.80,2053.95,2051.80,2053.95,2997
2019-04-08 09:59:00,2051.15,2052.00,2050.95,2051.95,2272


# Filter Stocks - portfolio maker

In [20]:
nifty50 = pd.read_csv("data/ind_nifty50list.csv")
niftynext50 = pd.read_csv("data/ind_niftynext50list.csv")
midcap50 = pd.read_csv("data/ind_niftymidcap50list.csv")

portfolio = nifty50.Symbol
industry = niftynext50.Industry.unique()

In [32]:
instruments_df.loc[['HDFC','SBIN','TCS','RIIL','BHARTIARTL','ADANIPORTS','DRREDDY'],'instrument_token']

tradingsymbol
HDFC           340481
SBIN           779521
TCS           2953217
RIIL           745473
BHARTIARTL    2714625
ADANIPORTS    3861249
DRREDDY        225537
Name: instrument_token, dtype: int64

In [33]:
portfolio = [340481,779521,2953217,745473, 2714625,3861249,225537]

portfolio

[340481, 779521, 2953217, 745473, 2714625, 3861249, 225537]

In [11]:
instruments_df.columns

Index(['instrument_token', 'exchange_token', 'name', 'last_price', 'expiry',
       'strike', 'tick_size', 'lot_size', 'instrument_type', 'segment',
       'exchange'],
      dtype='object')

In [12]:
instruments_df.loc['HDFC','instrument_token']

340481

# Live Data resampling

# Indicators Initialization

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50

yMin = raw_data.iloc[-1*noofcandles:-1]['low'].min()-10
yMax = raw_data.iloc[-1*noofcandles:-1]['high'].max()
xMin = raw_data.index[-1*noofcandles]
xMax = raw_data.index[-1]

# Algorithm

In [14]:
%run "KiteConnect_Library.ipynb" 

fig = createPlot()
raw_data = candlestick(raw_data,1,True)
raw_data = bbands(raw_data,1, True)

raw_data = macd(raw_data,3,True)
raw_data = rsi(raw_data,4, True)
raw_data = aroon(raw_data,5, True)
obv(raw_data,2, True)
#pivotPoint(raw_data, pos=1, plot=False)
#raw_data = calculateStats(raw_data)
#raw_data = emasma(raw_data)
#raw_data = adx(raw_data)
#raw_data = detectCDPattern(raw_data,strPlot="hammer", plot=True)
#raw_data = stoch(raw_data)

# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
VOLUME = 0
BBT = 0
BBM = 0
BBB = 0
AroonUp = 0
AroonDown = 0

UP = 0
DOWN = 1

# Methods
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a, days: ROC(a,days) >= 0.01
TREND_DOWN = lambda a, days: ROC(a,days) <= -0.01
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)

class algoTrade:
    
    def __init__(s, price):
        global OPEN, CLOSE, HIGH, LOW, VOLUME, BBT, BBM, BBB, AroonDown, AroonUp
        OPEN = price['open']
        CLOSE = price['close']
        HIGH = price['high']
        LOW = price['low']
        VOLUME = price['volume']
        BBT, BBM, BBB = BBANDS( CLOSE, 20,2,2,1)
        AroonDown, AroonUp = AROON(HIGH, LOW, 25)
    
    # Long Strategies
    def long_bb(self):
        return pd.DataFrame( (REF(CLOSE, 1) < REF(BBB, 1)) & (CLOSE > BBB), columns=["buy"] )
    
    def long_bull_engulf_ema(self):
        return pd.DataFrame( (EMA(CLOSE, 9) < EMA(CLOSE, 21)) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100) ,columns =["buy"])
    
    def long_hammer(self):
        return pd.DataFrame(TREND_DOWN(CLOSE, 10) & (CDLHAMMER(OPEN, HIGH, LOW, CLOSE) != 0), columns=["buy"])
    
    def long_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(A,B) ,columns=["buy"])
    
    def long_aroon(self):
        return pd.DataFrame((AroonUp > 50) & (AroonDown<50), columns=["buy"])
        
    
    # Short Strategies
    def short_bb(self):
        return pd.DataFrame((REF(CLOSE,1) > REF(BBT,1)) & (CLOSE<BBT), columns=["sell"])
    
    def short_bear_engulf(self):
        return pd.DataFrame(TREND_UP(CLOSE,10) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100),columns=["sell"])
    
    def short_hanging_man(self):
        return pd.DataFrame(
            TREND_UP(CLOSE, 10) &
            (CDLHANGINGMAN(OPEN, HIGH, LOW, CLOSE) == -100)
            , columns=["sell"])
    
    def short_bear_kicking(self):
        return pd.DataFrame(CDLKICKING(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
        
    def short_shooting_star(self):
        return pd.DataFrame( TREND_UP(CLOSE, 5) & CDLSHOOTINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
    
    def short_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(B,A) ,columns=["sell"])
    
    def short_aroon(self):
        return pd.DataFrame((AroonUp < 50) & (AroonDown > 50), columns=["sell"])
    
    def tradeDecision(self):
        buy  = self.long_bull_engulf_ema() | self.long_bb() | self.long_ichimoku()
        #buy = self.long_aroon()
        sell = self.short_bear_engulf() | self.short_bb() | self.short_hanging_man()
        #sell = self.short_aroon()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = sell['close'] = CLOSE

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        return (buy,sell)
        
a = algoTrade(raw_data)
buy, sell = a.tradeDecision()
annotateBuySell(buy, "Buy")
annotateBuySell(sell, "Sell")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



,sell,high,close
date,,,
2019-04-05 10:11:00,True,2054.85,2054.70
2019-04-05 10:13:00,True,2056.00,2054.50
2019-04-05 10:52:00,True,2055.00,2054.95
2019-04-05 10:59:00,True,2055.00,2054.95
2019-04-05 11:06:00,True,2059.85,2059.15
2019-04-05 11:08:00,True,2063.00,2060.95
2019-04-05 11:13:00,True,2060.65,2058.65
2019-04-05 11:37:00,True,2056.80,2056.70
2019-04-05 11:52:00,True,2055.30,2054.00


# Order Management

In [15]:
def placeorder(): 
    
    if False:
        # Place an order
        try:
            order_id = kite.place_order(tradingsymbol="INFY",
                                        exchange=kite.EXCHANGE_NSE,
                                        transaction_type=kite.TRANSACTION_TYPE_BUY,
                                        quantity=1,
                                        order_type=kite.ORDER_TYPE_MARKET,
                                        product=kite.PRODUCT_NRML)

            logging.info("Order placed. ID is: {}".format(order_id))
        except Exception as e:
            logging.info("Order placement failed: {}".format(e.message))


    
# Fetch all orders
kite.orders()



[]

# WebSocket: Live Tick Handler

In [50]:
import time

live_data = raw_data.iloc[-30:-1][['open','high','low','close','volume']]


import pickle
f = open('live_data_hdfc_full_4_8_2019.txt', 'wb')

count = 0
def ticksHandler(ticks):
    global live_data, f, count
    pickle.dump(ticks, f)
    
    count = count + 1
    
    if count <= 10:
        print(ticks)
    #logging.debug("Ticks: ticksHandler")
    
    #data = pd.DataFrame()
    #for tick in ticks:
    #    df = tick['ohlc']
    #    df['date'] = dt.datetime.now().replace(second=0, microsecond=0)
    #    df['volume'] = tick['volume']
    #    df['instrument_token'] = tick['instrument_token']

        #data = data.append(df, ignore_index=True)
        #raw_data.to_hdf('kite_data/live_data_hdfc.h5', key=symbol+keyToday,mode='a')
        #print(data.tail(1))
        #print(data)
        #print(" ")
        #print(" ")

    #data.set_index(['instrument_token','date'], inplace=True)
    
    #live_data = live_data.append(data.loc[340481]).copy(deep=True) 
    #freq="1min"
    #resampled_data = live_data.resample(freq).agg({'open':{'open':'first'},'high':{'high':'max'},'low':{'low':'min'},'close':{'close':'last'},'volume':{'volume':'sum'}})
    #resampled_data.dropna(inplace=True)
    #resampled_data.columns = resampled_data.columns.droplevel()
    #a = algoTrade(resampled_data)
    #buy, sell = a.tradeDecision()
    
    #print(buy)
    #print(sell)

In [51]:
#!python
import logging
import multiprocessing
from kiteconnect import KiteTicker
import copy 

logging.basicConfig(level=logging.CRITICAL)

jobs = []

#if mode == "Live":
# Initialise
kws = KiteTicker(KiteAPIKey, kite.access_token)

def on_ticks(ws, ticks):
    # Callback to receive ticks.
    #logging.debug("Ticks: {}".format(ticks))
    #count = count + 1
    
    #print(ticks)
    ticksHandler(ticks)
    #jobs.append(p)
    #p.start()


def on_connect(ws, response):
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe(portfolio)

    # Set RELIANCE to tick in `full` mode.
    # MODE_LTP, MODE_QUOTE, or MODE_FULL
    
    ws.set_mode(ws.MODE_FULL, [340481, 2953217, 745473]) 
    #ws.set_mode(ws.MODE_LTP, [225537, 3861249]) 
    #ws.set_mode(ws.MODE_MODE_QUOTE, [2714625,779521]) 

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
#kws.on_close = on_close

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.

#p = multiprocessing.Process(target=kws.connect)
#jobs.append(p)
#p.start()


In [52]:
kws.connect(threaded=True)

[{'tradable': True, 'mode': 'quote', 'instrument_token': 225537, 'last_price': 2762.1, 'last_quantity': 1, 'average_price': 2766.41, 'volume': 94835, 'buy_quantity': 54745, 'sell_quantity': 60966, 'ohlc': {'open': 2760.0, 'high': 2780.0, 'low': 2752.25, 'close': 2754.35}, 'change': 0.28137310073157007}, {'tradable': True, 'mode': 'quote', 'instrument_token': 779521, 'last_price': 317.65, 'last_quantity': 135, 'average_price': 317.52, 'volume': 3928653, 'buy_quantity': 1407649, 'sell_quantity': 3003899, 'ohlc': {'open': 318.25, 'high': 319.25, 'low': 315.35, 'close': 317.05}, 'change': 0.18924459864373627}, {'tradable': True, 'mode': 'quote', 'instrument_token': 3861249, 'last_price': 380.9, 'last_quantity': 43, 'average_price': 382.6, 'volume': 555807, 'buy_quantity': 293804, 'sell_quantity': 1266317, 'ohlc': {'open': 388.3, 'high': 388.3, 'low': 380.0, 'close': 387.15}, 'change': -1.6143613586465195}, {'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.1, 

[{'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.1, 'last_quantity': 90, 'average_price': 358.17, 'volume': 1737384, 'buy_quantity': 492405, 'sell_quantity': 757744, 'ohlc': {'open': 326.2, 'high': 368.95, 'low': 326.2, 'close': 324.65}, 'change': 11.227475743107977, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 47), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 47), 'depth': {'buy': [{'quantity': 924, 'price': 361.05, 'orders': 1}, {'quantity': 1768, 'price': 361.0, 'orders': 15}, {'quantity': 18, 'price': 360.95, 'orders': 3}, {'quantity': 232, 'price': 360.9, 'orders': 3}, {'quantity': 108, 'price': 360.85, 'orders': 2}], 'sell': [{'quantity': 337, 'price': 361.1, 'orders': 2}, {'quantity': 129, 'price': 361.55, 'orders': 2}, {'quantity': 155, 'price': 361.6, 'orders': 2}, {'quantity': 21, 'price': 361.7, 'orders': 2}, {'quantity': 254, 'price': 361.75, 'orders': 4}]}}, {'tradable': True, 'm

[{'tradable': True, 'mode': 'full', 'instrument_token': 340481, 'last_price': 2050.4, 'last_quantity': 1, 'average_price': 2053.2, 'volume': 299967, 'buy_quantity': 121886, 'sell_quantity': 398103, 'ohlc': {'open': 2068.0, 'high': 2072.5, 'low': 2043.55, 'close': 2059.2}, 'change': -0.4273504273504141, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 48), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 49), 'depth': {'buy': [{'quantity': 9, 'price': 2050.25, 'orders': 1}, {'quantity': 5, 'price': 2050.2, 'orders': 1}, {'quantity': 1, 'price': 2050.1, 'orders': 1}, {'quantity': 817, 'price': 2050.0, 'orders': 20}, {'quantity': 1, 'price': 2049.65, 'orders': 1}], 'sell': [{'quantity': 27, 'price': 2050.4, 'orders': 2}, {'quantity': 20, 'price': 2050.45, 'orders': 1}, {'quantity': 96, 'price': 2050.5, 'orders': 3}, {'quantity': 4, 'price': 2050.55, 'orders': 1}, {'quantity': 4, 'price': 2050.7, 'orders': 1}]}}, {'tradable': True, 'mod

[{'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.05, 'last_quantity': 100, 'average_price': 358.17, 'volume': 1737835, 'buy_quantity': 491954, 'sell_quantity': 757341, 'ohlc': {'open': 326.2, 'high': 368.95, 'low': 326.2, 'close': 324.65}, 'change': 11.212074541814273, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 49), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 51), 'depth': {'buy': [{'quantity': 823, 'price': 361.05, 'orders': 1}, {'quantity': 1768, 'price': 361.0, 'orders': 15}, {'quantity': 18, 'price': 360.95, 'orders': 3}, {'quantity': 232, 'price': 360.9, 'orders': 3}, {'quantity': 108, 'price': 360.85, 'orders': 2}], 'sell': [{'quantity': 4, 'price': 361.1, 'orders': 1}, {'quantity': 77, 'price': 361.4, 'orders': 1}, {'quantity': 20, 'price': 361.45, 'orders': 1}, {'quantity': 4, 'price': 361.6, 'orders': 1}, {'quantity': 1, 'price': 361.7, 'orders': 1}]}}, {'tradable': True, 'mode': 

In [277]:
kws.close()
f.close()

In [278]:
kws.is_connected()

False

In [183]:
jobs[0].terminate()

In [279]:
import datetime
sample_tick = [{'tradable': True, 'mode': 'quote', 'instrument_token': 225537, 'last_price': 2762.1, 'last_quantity': 1, 'average_price': 2766.41, 'volume': 94835, 'buy_quantity': 54745, 'sell_quantity': 60966, 'ohlc': {'open': 2760.0, 'high': 2780.0, 'low': 2752.25, 'close': 2754.35}, 'change': 0.28137310073157007}, {'tradable': True, 'mode': 'quote', 'instrument_token': 779521, 'last_price': 317.65, 'last_quantity': 135, 'average_price': 317.52, 'volume': 3928653, 'buy_quantity': 1407649, 'sell_quantity': 3003899, 'ohlc': {'open': 318.25, 'high': 319.25, 'low': 315.35, 'close': 317.05}, 'change': 0.18924459864373627}, {'tradable': True, 'mode': 'quote', 'instrument_token': 3861249, 'last_price': 380.9, 'last_quantity': 43, 'average_price': 382.6, 'volume': 555807, 'buy_quantity': 293804, 'sell_quantity': 1266317, 'ohlc': {'open': 388.3, 'high': 388.3, 'low': 380.0, 'close': 387.15}, 'change': -1.6143613586465195}, {'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.1, 'last_quantity': 77, 'average_price': 358.17, 'volume': 1737294, 'buy_quantity': 492429, 'sell_quantity': 758779, 'ohlc': {'open': 326.2, 'high': 368.95, 'low': 326.2, 'close': 324.65}, 'change': 11.227475743107977, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 43), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 43), 'depth': {'buy': [{'quantity': 924, 'price': 361.05, 'orders': 1}, {'quantity': 1768, 'price': 361.0, 'orders': 15}, {'quantity': 18, 'price': 360.95, 'orders': 3}, {'quantity': 232, 'price': 360.9, 'orders': 3}, {'quantity': 108, 'price': 360.85, 'orders': 2}], 'sell': [{'quantity': 423, 'price': 361.1, 'orders': 1}, {'quantity': 129, 'price': 361.55, 'orders': 2}, {'quantity': 155, 'price': 361.6, 'orders': 2}, {'quantity': 20, 'price': 361.7, 'orders': 1}, {'quantity': 254, 'price': 361.75, 'orders': 4}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 340481, 'last_price': 2050.35, 'last_quantity': 12, 'average_price': 2053.2, 'volume': 299942, 'buy_quantity': 121688, 'sell_quantity': 398644, 'ohlc': {'open': 2068.0, 'high': 2072.5, 'low': 2043.55, 'close': 2059.2}, 'change': -0.4297785547785504, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 45), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 45), 'depth': {'buy': [{'quantity': 26, 'price': 2050.35, 'orders': 2}, {'quantity': 5, 'price': 2050.2, 'orders': 1}, {'quantity': 1, 'price': 2050.1, 'orders': 1}, {'quantity': 14, 'price': 2050.05, 'orders': 1}, {'quantity': 804, 'price': 2050.0, 'orders': 19}], 'sell': [{'quantity': 107, 'price': 2050.5, 'orders': 3}, {'quantity': 12, 'price': 2050.55, 'orders': 2}, {'quantity': 6, 'price': 2050.7, 'orders': 2}, {'quantity': 3, 'price': 2050.85, 'orders': 1}, {'quantity': 14, 'price': 2050.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 2953217, 'last_price': 2042.65, 'last_quantity': 8, 'average_price': 2047.71, 'volume': 592121, 'buy_quantity': 159570, 'sell_quantity': 360821, 'ohlc': {'open': 2059.0, 'high': 2063.75, 'low': 2032.7, 'close': 2048.3}, 'change': -0.2758385002196988, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 44), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 45), 'depth': {'buy': [{'quantity': 7, 'price': 2042.65, 'orders': 1}, {'quantity': 13, 'price': 2042.35, 'orders': 1}, {'quantity': 1, 'price': 2042.15, 'orders': 1}, {'quantity': 45, 'price': 2042.1, 'orders': 3}, {'quantity': 122, 'price': 2042.05, 'orders': 3}], 'sell': [{'quantity': 47, 'price': 2042.7, 'orders': 4}, {'quantity': 71, 'price': 2042.75, 'orders': 3}, {'quantity': 10, 'price': 2042.8, 'orders': 1}, {'quantity': 10, 'price': 2042.85, 'orders': 1}, {'quantity': 11, 'price': 2042.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'quote', 'instrument_token': 2714625, 'last_price': 358.4, 'last_quantity': 50, 'average_price': 360.74, 'volume': 2027746, 'buy_quantity': 378121, 'sell_quantity': 1410520, 'ohlc': {'open': 358.4, 'high': 363.8, 'low': 357.1, 'close': 356.65}, 'change': 0.49067713444553485}]

sample_ticks = [[{'tradable': True, 'mode': 'quote', 'instrument_token': 225537, 'last_price': 2762.1, 'last_quantity': 1, 'average_price': 2766.41, 'volume': 94835, 'buy_quantity': 54745, 'sell_quantity': 60966, 'ohlc': {'open': 2760.0, 'high': 2780.0, 'low': 2752.25, 'close': 2754.35}, 'change': 0.28137310073157007}, {'tradable': True, 'mode': 'quote', 'instrument_token': 779521, 'last_price': 317.65, 'last_quantity': 135, 'average_price': 317.52, 'volume': 3928653, 'buy_quantity': 1407649, 'sell_quantity': 3003899, 'ohlc': {'open': 318.25, 'high': 319.25, 'low': 315.35, 'close': 317.05}, 'change': 0.18924459864373627}, {'tradable': True, 'mode': 'quote', 'instrument_token': 3861249, 'last_price': 380.9, 'last_quantity': 43, 'average_price': 382.6, 'volume': 555807, 'buy_quantity': 293804, 'sell_quantity': 1266317, 'ohlc': {'open': 388.3, 'high': 388.3, 'low': 380.0, 'close': 387.15}, 'change': -1.6143613586465195}, {'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.1, 'last_quantity': 77, 'average_price': 358.17, 'volume': 1737294, 'buy_quantity': 492429, 'sell_quantity': 758779, 'ohlc': {'open': 326.2, 'high': 368.95, 'low': 326.2, 'close': 324.65}, 'change': 11.227475743107977, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 43), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 43), 'depth': {'buy': [{'quantity': 924, 'price': 361.05, 'orders': 1}, {'quantity': 1768, 'price': 361.0, 'orders': 15}, {'quantity': 18, 'price': 360.95, 'orders': 3}, {'quantity': 232, 'price': 360.9, 'orders': 3}, {'quantity': 108, 'price': 360.85, 'orders': 2}], 'sell': [{'quantity': 423, 'price': 361.1, 'orders': 1}, {'quantity': 129, 'price': 361.55, 'orders': 2}, {'quantity': 155, 'price': 361.6, 'orders': 2}, {'quantity': 20, 'price': 361.7, 'orders': 1}, {'quantity': 254, 'price': 361.75, 'orders': 4}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 340481, 'last_price': 2050.35, 'last_quantity': 12, 'average_price': 2053.2, 'volume': 299942, 'buy_quantity': 121688, 'sell_quantity': 398644, 'ohlc': {'open': 2068.0, 'high': 2072.5, 'low': 2043.55, 'close': 2059.2}, 'change': -0.4297785547785504, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 45), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 45), 'depth': {'buy': [{'quantity': 26, 'price': 2050.35, 'orders': 2}, {'quantity': 5, 'price': 2050.2, 'orders': 1}, {'quantity': 1, 'price': 2050.1, 'orders': 1}, {'quantity': 14, 'price': 2050.05, 'orders': 1}, {'quantity': 804, 'price': 2050.0, 'orders': 19}], 'sell': [{'quantity': 107, 'price': 2050.5, 'orders': 3}, {'quantity': 12, 'price': 2050.55, 'orders': 2}, {'quantity': 6, 'price': 2050.7, 'orders': 2}, {'quantity': 3, 'price': 2050.85, 'orders': 1}, {'quantity': 14, 'price': 2050.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 2953217, 'last_price': 2042.65, 'last_quantity': 8, 'average_price': 2047.71, 'volume': 592121, 'buy_quantity': 159570, 'sell_quantity': 360821, 'ohlc': {'open': 2059.0, 'high': 2063.75, 'low': 2032.7, 'close': 2048.3}, 'change': -0.2758385002196988, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 44), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 45), 'depth': {'buy': [{'quantity': 7, 'price': 2042.65, 'orders': 1}, {'quantity': 13, 'price': 2042.35, 'orders': 1}, {'quantity': 1, 'price': 2042.15, 'orders': 1}, {'quantity': 45, 'price': 2042.1, 'orders': 3}, {'quantity': 122, 'price': 2042.05, 'orders': 3}], 'sell': [{'quantity': 47, 'price': 2042.7, 'orders': 4}, {'quantity': 71, 'price': 2042.75, 'orders': 3}, {'quantity': 10, 'price': 2042.8, 'orders': 1}, {'quantity': 10, 'price': 2042.85, 'orders': 1}, {'quantity': 11, 'price': 2042.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'quote', 'instrument_token': 2714625, 'last_price': 358.4, 'last_quantity': 50, 'average_price': 360.74, 'volume': 2027746, 'buy_quantity': 378121, 'sell_quantity': 1410520, 'ohlc': {'open': 358.4, 'high': 363.8, 'low': 357.1, 'close': 356.65}, 'change': 0.49067713444553485}],
[{'tradable': True, 'mode': 'quote', 'instrument_token': 779521, 'last_price': 317.65, 'last_quantity': 135, 'average_price': 317.52, 'volume': 3928653, 'buy_quantity': 1407649, 'sell_quantity': 3004630, 'ohlc': {'open': 318.25, 'high': 319.25, 'low': 315.35, 'close': 317.05}, 'change': 0.18924459864373627}, {'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.1, 'last_quantity': 77, 'average_price': 358.17, 'volume': 1737294, 'buy_quantity': 492365, 'sell_quantity': 758829, 'ohlc': {'open': 326.2, 'high': 368.95, 'low': 326.2, 'close': 324.65}, 'change': 11.227475743107977, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 43), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 46), 'depth': {'buy': [{'quantity': 924, 'price': 361.05, 'orders': 1}, {'quantity': 1768, 'price': 361.0, 'orders': 15}, {'quantity': 18, 'price': 360.95, 'orders': 3}, {'quantity': 232, 'price': 360.9, 'orders': 3}, {'quantity': 108, 'price': 360.85, 'orders': 2}], 'sell': [{'quantity': 423, 'price': 361.1, 'orders': 1}, {'quantity': 129, 'price': 361.55, 'orders': 2}, {'quantity': 155, 'price': 361.6, 'orders': 2}, {'quantity': 20, 'price': 361.7, 'orders': 1}, {'quantity': 254, 'price': 361.75, 'orders': 4}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 340481, 'last_price': 2050.5, 'last_quantity': 11, 'average_price': 2053.2, 'volume': 299942, 'buy_quantity': 121688, 'sell_quantity': 398644, 'ohlc': {'open': 2068.0, 'high': 2072.5, 'low': 2043.55, 'close': 2059.2}, 'change': -0.4224941724941637, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 45), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 46), 'depth': {'buy': [{'quantity': 26, 'price': 2050.35, 'orders': 2}, {'quantity': 5, 'price': 2050.2, 'orders': 1}, {'quantity': 1, 'price': 2050.1, 'orders': 1}, {'quantity': 14, 'price': 2050.05, 'orders': 1}, {'quantity': 804, 'price': 2050.0, 'orders': 19}], 'sell': [{'quantity': 107, 'price': 2050.5, 'orders': 3}, {'quantity': 12, 'price': 2050.55, 'orders': 2}, {'quantity': 6, 'price': 2050.7, 'orders': 2}, {'quantity': 3, 'price': 2050.85, 'orders': 1}, {'quantity': 14, 'price': 2050.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 2953217, 'last_price': 2042.7, 'last_quantity': 3, 'average_price': 2047.71, 'volume': 592121, 'buy_quantity': 159570, 'sell_quantity': 360821, 'ohlc': {'open': 2059.0, 'high': 2063.75, 'low': 2032.7, 'close': 2048.3}, 'change': -0.27339745154519046, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 44), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 46), 'depth': {'buy': [{'quantity': 7, 'price': 2042.65, 'orders': 1}, {'quantity': 13, 'price': 2042.35, 'orders': 1}, {'quantity': 1, 'price': 2042.15, 'orders': 1}, {'quantity': 45, 'price': 2042.1, 'orders': 3}, {'quantity': 122, 'price': 2042.05, 'orders': 3}], 'sell': [{'quantity': 47, 'price': 2042.7, 'orders': 4}, {'quantity': 71, 'price': 2042.75, 'orders': 3}, {'quantity': 10, 'price': 2042.8, 'orders': 1}, {'quantity': 10, 'price': 2042.85, 'orders': 1}, {'quantity': 11, 'price': 2042.9, 'orders': 1}]}}],
[{'tradable': True, 'mode': 'quote', 'instrument_token': 3861249, 'last_price': 380.9, 'last_quantity': 43, 'average_price': 382.6, 'volume': 555807, 'buy_quantity': 294215, 'sell_quantity': 1266317, 'ohlc': {'open': 388.3, 'high': 388.3, 'low': 380.0, 'close': 387.15}, 'change': -1.6143613586465195}, {'tradable': True, 'mode': 'full', 'instrument_token': 340481, 'last_price': 2050.5, 'last_quantity': 11, 'average_price': 2053.2, 'volume': 299953, 'buy_quantity': 121694, 'sell_quantity': 398105, 'ohlc': {'open': 2068.0, 'high': 2072.5, 'low': 2043.55, 'close': 2059.2}, 'change': -0.4224941724941637, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 45), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 46), 'depth': {'buy': [{'quantity': 19, 'price': 2050.25, 'orders': 1}, {'quantity': 5, 'price': 2050.2, 'orders': 1}, {'quantity': 1, 'price': 2050.1, 'orders': 1}, {'quantity': 14, 'price': 2050.05, 'orders': 1}, {'quantity': 817, 'price': 2050.0, 'orders': 20}], 'sell': [{'quantity': 101, 'price': 2050.5, 'orders': 4}, {'quantity': 12, 'price': 2050.55, 'orders': 2}, {'quantity': 4, 'price': 2050.7, 'orders': 1}, {'quantity': 4, 'price': 2050.85, 'orders': 2}, {'quantity': 14, 'price': 2050.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 2953217, 'last_price': 2042.7, 'last_quantity': 1, 'average_price': 2047.71, 'volume': 592125, 'buy_quantity': 159830, 'sell_quantity': 360786, 'ohlc': {'open': 2059.0, 'high': 2063.75, 'low': 2032.7, 'close': 2048.3}, 'change': -0.27339745154519046, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 46), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 46), 'depth': {'buy': [{'quantity': 7, 'price': 2042.65, 'orders': 1}, {'quantity': 23, 'price': 2042.55, 'orders': 2}, {'quantity': 1, 'price': 2042.15, 'orders': 1}, {'quantity': 45, 'price': 2042.1, 'orders': 3}, {'quantity': 122, 'price': 2042.05, 'orders': 3}], 'sell': [{'quantity': 43, 'price': 2042.7, 'orders': 4}, {'quantity': 71, 'price': 2042.75, 'orders': 3}, {'quantity': 10, 'price': 2042.8, 'orders': 1}, {'quantity': 10, 'price': 2042.85, 'orders': 1}, {'quantity': 11, 'price': 2042.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'quote', 'instrument_token': 2714625, 'last_price': 358.4, 'last_quantity': 50, 'average_price': 360.74, 'volume': 2027746, 'buy_quantity': 378646, 'sell_quantity': 1412570, 'ohlc': {'open': 358.4, 'high': 363.8, 'low': 357.1, 'close': 356.65}, 'change': 0.49067713444553485}, {'tradable': True, 'mode': 'quote', 'instrument_token': 225537, 'last_price': 2762.1, 'last_quantity': 1, 'average_price': 2766.41, 'volume': 94835, 'buy_quantity': 54758, 'sell_quantity': 60966, 'ohlc': {'open': 2760.0, 'high': 2780.0, 'low': 2752.25, 'close': 2754.35}, 'change': 0.28137310073157007}],
[{'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.1, 'last_quantity': 90, 'average_price': 358.17, 'volume': 1737384, 'buy_quantity': 492405, 'sell_quantity': 757744, 'ohlc': {'open': 326.2, 'high': 368.95, 'low': 326.2, 'close': 324.65}, 'change': 11.227475743107977, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 47), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 47), 'depth': {'buy': [{'quantity': 924, 'price': 361.05, 'orders': 1}, {'quantity': 1768, 'price': 361.0, 'orders': 15}, {'quantity': 18, 'price': 360.95, 'orders': 3}, {'quantity': 232, 'price': 360.9, 'orders': 3}, {'quantity': 108, 'price': 360.85, 'orders': 2}], 'sell': [{'quantity': 337, 'price': 361.1, 'orders': 2}, {'quantity': 129, 'price': 361.55, 'orders': 2}, {'quantity': 155, 'price': 361.6, 'orders': 2}, {'quantity': 21, 'price': 361.7, 'orders': 2}, {'quantity': 254, 'price': 361.75, 'orders': 4}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 340481, 'last_price': 2050.5, 'last_quantity': 3, 'average_price': 2053.2, 'volume': 299953, 'buy_quantity': 121694, 'sell_quantity': 398105, 'ohlc': {'open': 2068.0, 'high': 2072.5, 'low': 2043.55, 'close': 2059.2}, 'change': -0.4224941724941637, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 45), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 47), 'depth': {'buy': [{'quantity': 19, 'price': 2050.25, 'orders': 1}, {'quantity': 5, 'price': 2050.2, 'orders': 1}, {'quantity': 1, 'price': 2050.1, 'orders': 1}, {'quantity': 14, 'price': 2050.05, 'orders': 1}, {'quantity': 817, 'price': 2050.0, 'orders': 20}], 'sell': [{'quantity': 101, 'price': 2050.5, 'orders': 4}, {'quantity': 12, 'price': 2050.55, 'orders': 2}, {'quantity': 4, 'price': 2050.7, 'orders': 1}, {'quantity': 4, 'price': 2050.85, 'orders': 2}, {'quantity': 14, 'price': 2050.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 2953217, 'last_price': 2042.7, 'last_quantity': 4, 'average_price': 2047.71, 'volume': 592147, 'buy_quantity': 159580, 'sell_quantity': 360806, 'ohlc': {'open': 2059.0, 'high': 2063.75, 'low': 2032.7, 'close': 2048.3}, 'change': -0.27339745154519046, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 46), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 47), 'depth': {'buy': [{'quantity': 7, 'price': 2042.65, 'orders': 1}, {'quantity': 23, 'price': 2042.55, 'orders': 2}, {'quantity': 1, 'price': 2042.15, 'orders': 1}, {'quantity': 45, 'price': 2042.1, 'orders': 3}, {'quantity': 122, 'price': 2042.05, 'orders': 3}], 'sell': [{'quantity': 43, 'price': 2042.7, 'orders': 4}, {'quantity': 71, 'price': 2042.75, 'orders': 3}, {'quantity': 10, 'price': 2042.8, 'orders': 1}, {'quantity': 10, 'price': 2042.85, 'orders': 1}, {'quantity': 11, 'price': 2042.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'quote', 'instrument_token': 2714625, 'last_price': 358.4, 'last_quantity': 50, 'average_price': 360.74, 'volume': 2027746, 'buy_quantity': 378949, 'sell_quantity': 1412570, 'ohlc': {'open': 358.4, 'high': 363.8, 'low': 357.1, 'close': 356.65}, 'change': 0.49067713444553485}, {'tradable': True, 'mode': 'quote', 'instrument_token': 779521, 'last_price': 317.65, 'last_quantity': 135, 'average_price': 317.52, 'volume': 3928653, 'buy_quantity': 1414255, 'sell_quantity': 2999865, 'ohlc': {'open': 318.25, 'high': 319.25, 'low': 315.35, 'close': 317.05}, 'change': 0.18924459864373627}, {'tradable': True, 'mode': 'quote', 'instrument_token': 3861249, 'last_price': 380.9, 'last_quantity': 100, 'average_price': 382.6, 'volume': 555807, 'buy_quantity': 294215, 'sell_quantity': 1266317, 'ohlc': {'open': 388.3, 'high': 388.3, 'low': 380.0, 'close': 387.15}, 'change': -1.6143613586465195}],
[{'tradable': True, 'mode': 'quote', 'instrument_token': 779521, 'last_price': 317.5, 'last_quantity': 100, 'average_price': 317.52, 'volume': 3928653, 'buy_quantity': 1414255, 'sell_quantity': 2999865, 'ohlc': {'open': 318.25, 'high': 319.25, 'low': 315.35, 'close': 317.05}, 'change': 0.1419334489828067}, {'tradable': True, 'mode': 'quote', 'instrument_token': 3861249, 'last_price': 380.85, 'last_quantity': 51, 'average_price': 382.6, 'volume': 556127, 'buy_quantity': 293754, 'sell_quantity': 1267097, 'ohlc': {'open': 388.3, 'high': 388.3, 'low': 380.0, 'close': 387.15}, 'change': -1.6272762495156798}, {'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.1, 'last_quantity': 100, 'average_price': 358.17, 'volume': 1737384, 'buy_quantity': 492405, 'sell_quantity': 757744, 'ohlc': {'open': 326.2, 'high': 368.95, 'low': 326.2, 'close': 324.65}, 'change': 11.227475743107977, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 47), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 48), 'depth': {'buy': [{'quantity': 924, 'price': 361.05, 'orders': 1}, {'quantity': 1768, 'price': 361.0, 'orders': 15}, {'quantity': 18, 'price': 360.95, 'orders': 3}, {'quantity': 232, 'price': 360.9, 'orders': 3}, {'quantity': 108, 'price': 360.85, 'orders': 2}], 'sell': [{'quantity': 337, 'price': 361.1, 'orders': 2}, {'quantity': 129, 'price': 361.55, 'orders': 2}, {'quantity': 155, 'price': 361.6, 'orders': 2}, {'quantity': 21, 'price': 361.7, 'orders': 2}, {'quantity': 254, 'price': 361.75, 'orders': 4}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 340481, 'last_price': 2050.5, 'last_quantity': 3, 'average_price': 2053.2, 'volume': 299966, 'buy_quantity': 121885, 'sell_quantity': 398057, 'ohlc': {'open': 2068.0, 'high': 2072.5, 'low': 2043.55, 'close': 2059.2}, 'change': -0.4224941724941637, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 46), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 47), 'depth': {'buy': [{'quantity': 9, 'price': 2050.25, 'orders': 1}, {'quantity': 5, 'price': 2050.2, 'orders': 1}, {'quantity': 1, 'price': 2050.1, 'orders': 1}, {'quantity': 14, 'price': 2050.05, 'orders': 1}, {'quantity': 817, 'price': 2050.0, 'orders': 20}], 'sell': [{'quantity': 20, 'price': 2050.45, 'orders': 1}, {'quantity': 105, 'price': 2050.5, 'orders': 5}, {'quantity': 12, 'price': 2050.55, 'orders': 2}, {'quantity': 4, 'price': 2050.7, 'orders': 1}, {'quantity': 1, 'price': 2050.8, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 2953217, 'last_price': 2042.7, 'last_quantity': 4, 'average_price': 2047.71, 'volume': 592151, 'buy_quantity': 159641, 'sell_quantity': 360814, 'ohlc': {'open': 2059.0, 'high': 2063.75, 'low': 2032.7, 'close': 2048.3}, 'change': -0.27339745154519046, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 47), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 48), 'depth': {'buy': [{'quantity': 78, 'price': 2042.65, 'orders': 2}, {'quantity': 23, 'price': 2042.55, 'orders': 2}, {'quantity': 1, 'price': 2042.15, 'orders': 1}, {'quantity': 45, 'price': 2042.1, 'orders': 3}, {'quantity': 122, 'price': 2042.05, 'orders': 3}], 'sell': [{'quantity': 49, 'price': 2042.7, 'orders': 5}, {'quantity': 71, 'price': 2042.75, 'orders': 3}, {'quantity': 10, 'price': 2042.8, 'orders': 1}, {'quantity': 10, 'price': 2042.85, 'orders': 1}, {'quantity': 11, 'price': 2042.9, 'orders': 1}]}}],
[{'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.1, 'last_quantity': 100, 'average_price': 358.17, 'volume': 1737484, 'buy_quantity': 492305, 'sell_quantity': 757493, 'ohlc': {'open': 326.2, 'high': 368.95, 'low': 326.2, 'close': 324.65}, 'change': 11.227475743107977, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 48), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 48), 'depth': {'buy': [{'quantity': 924, 'price': 361.05, 'orders': 1}, {'quantity': 1768, 'price': 361.0, 'orders': 15}, {'quantity': 18, 'price': 360.95, 'orders': 3}, {'quantity': 232, 'price': 360.9, 'orders': 3}, {'quantity': 108, 'price': 360.85, 'orders': 2}], 'sell': [{'quantity': 237, 'price': 361.1, 'orders': 2}, {'quantity': 20, 'price': 361.45, 'orders': 1}, {'quantity': 77, 'price': 361.55, 'orders': 1}, {'quantity': 4, 'price': 361.6, 'orders': 1}, {'quantity': 53, 'price': 361.7, 'orders': 2}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 340481, 'last_price': 2050.4, 'last_quantity': 1, 'average_price': 2053.2, 'volume': 299966, 'buy_quantity': 121885, 'sell_quantity': 398057, 'ohlc': {'open': 2068.0, 'high': 2072.5, 'low': 2043.55, 'close': 2059.2}, 'change': -0.4273504273504141, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 46), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 48), 'depth': {'buy': [{'quantity': 9, 'price': 2050.25, 'orders': 1}, {'quantity': 5, 'price': 2050.2, 'orders': 1}, {'quantity': 1, 'price': 2050.1, 'orders': 1}, {'quantity': 14, 'price': 2050.05, 'orders': 1}, {'quantity': 817, 'price': 2050.0, 'orders': 20}], 'sell': [{'quantity': 20, 'price': 2050.45, 'orders': 1}, {'quantity': 105, 'price': 2050.5, 'orders': 5}, {'quantity': 12, 'price': 2050.55, 'orders': 2}, {'quantity': 4, 'price': 2050.7, 'orders': 1}, {'quantity': 1, 'price': 2050.8, 'orders': 1}]}}, {'tradable': True, 'mode': 'quote', 'instrument_token': 2714625, 'last_price': 358.4, 'last_quantity': 50, 'average_price': 360.74, 'volume': 2027746, 'buy_quantity': 378949, 'sell_quantity': 1412575, 'ohlc': {'open': 358.4, 'high': 363.8, 'low': 357.1, 'close': 356.65}, 'change': 0.49067713444553485}, {'tradable': True, 'mode': 'quote', 'instrument_token': 225537, 'last_price': 2762.1, 'last_quantity': 1, 'average_price': 2766.41, 'volume': 94835, 'buy_quantity': 54760, 'sell_quantity': 60966, 'ohlc': {'open': 2760.0, 'high': 2780.0, 'low': 2752.25, 'close': 2754.35}, 'change': 0.28137310073157007}, {'tradable': True, 'mode': 'quote', 'instrument_token': 779521, 'last_price': 317.5, 'last_quantity': 150, 'average_price': 317.52, 'volume': 3929066, 'buy_quantity': 1416748, 'sell_quantity': 2999701, 'ohlc': {'open': 318.25, 'high': 319.25, 'low': 315.35, 'close': 317.05}, 'change': 0.1419334489828067}, {'tradable': True, 'mode': 'quote', 'instrument_token': 3861249, 'last_price': 380.85, 'last_quantity': 51, 'average_price': 382.6, 'volume': 556178, 'buy_quantity': 293754, 'sell_quantity': 1266983, 'ohlc': {'open': 388.3, 'high': 388.3, 'low': 380.0, 'close': 387.15}, 'change': -1.6272762495156798}],
[{'tradable': True, 'mode': 'full', 'instrument_token': 340481, 'last_price': 2050.4, 'last_quantity': 1, 'average_price': 2053.2, 'volume': 299967, 'buy_quantity': 121886, 'sell_quantity': 398103, 'ohlc': {'open': 2068.0, 'high': 2072.5, 'low': 2043.55, 'close': 2059.2}, 'change': -0.4273504273504141, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 48), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 49), 'depth': {'buy': [{'quantity': 9, 'price': 2050.25, 'orders': 1}, {'quantity': 5, 'price': 2050.2, 'orders': 1}, {'quantity': 1, 'price': 2050.1, 'orders': 1}, {'quantity': 817, 'price': 2050.0, 'orders': 20}, {'quantity': 1, 'price': 2049.65, 'orders': 1}], 'sell': [{'quantity': 27, 'price': 2050.4, 'orders': 2}, {'quantity': 20, 'price': 2050.45, 'orders': 1}, {'quantity': 96, 'price': 2050.5, 'orders': 3}, {'quantity': 4, 'price': 2050.55, 'orders': 1}, {'quantity': 4, 'price': 2050.7, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 2953217, 'last_price': 2042.7, 'last_quantity': 4, 'average_price': 2047.71, 'volume': 592151, 'buy_quantity': 159593, 'sell_quantity': 360814, 'ohlc': {'open': 2059.0, 'high': 2063.75, 'low': 2032.7, 'close': 2048.3}, 'change': -0.27339745154519046, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 47), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 49), 'depth': {'buy': [{'quantity': 8, 'price': 2042.65, 'orders': 2}, {'quantity': 23, 'price': 2042.55, 'orders': 2}, {'quantity': 2, 'price': 2042.5, 'orders': 1}, {'quantity': 1, 'price': 2042.15, 'orders': 1}, {'quantity': 45, 'price': 2042.1, 'orders': 3}], 'sell': [{'quantity': 49, 'price': 2042.7, 'orders': 5}, {'quantity': 71, 'price': 2042.75, 'orders': 3}, {'quantity': 10, 'price': 2042.8, 'orders': 1}, {'quantity': 10, 'price': 2042.85, 'orders': 1}, {'quantity': 11, 'price': 2042.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'quote', 'instrument_token': 225537, 'last_price': 2762.1, 'last_quantity': 1, 'average_price': 2766.41, 'volume': 94835, 'buy_quantity': 54760, 'sell_quantity': 60955, 'ohlc': {'open': 2760.0, 'high': 2780.0, 'low': 2752.25, 'close': 2754.35}, 'change': 0.28137310073157007}, {'tradable': True, 'mode': 'quote', 'instrument_token': 779521, 'last_price': 317.5, 'last_quantity': 150, 'average_price': 317.52, 'volume': 3929066, 'buy_quantity': 1416748, 'sell_quantity': 2999701, 'ohlc': {'open': 318.25, 'high': 319.25, 'low': 315.35, 'close': 317.05}, 'change': 0.1419334489828067}, {'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.05, 'last_quantity': 1, 'average_price': 358.17, 'volume': 1737485, 'buy_quantity': 492304, 'sell_quantity': 757474, 'ohlc': {'open': 326.2, 'high': 368.95, 'low': 326.2, 'close': 324.65}, 'change': 11.212074541814273, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 49), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 49), 'depth': {'buy': [{'quantity': 923, 'price': 361.05, 'orders': 1}, {'quantity': 1768, 'price': 361.0, 'orders': 15}, {'quantity': 18, 'price': 360.95, 'orders': 3}, {'quantity': 232, 'price': 360.9, 'orders': 3}, {'quantity': 108, 'price': 360.85, 'orders': 2}], 'sell': [{'quantity': 237, 'price': 361.1, 'orders': 2}, {'quantity': 77, 'price': 361.4, 'orders': 1}, {'quantity': 20, 'price': 361.45, 'orders': 1}, {'quantity': 4, 'price': 361.6, 'orders': 1}, {'quantity': 53, 'price': 361.7, 'orders': 2}]}}],
[{'tradable': True, 'mode': 'quote', 'instrument_token': 779521, 'last_price': 317.5, 'last_quantity': 150, 'average_price': 317.52, 'volume': 3929066, 'buy_quantity': 1417117, 'sell_quantity': 2999951, 'ohlc': {'open': 318.25, 'high': 319.25, 'low': 315.35, 'close': 317.05}, 'change': 0.1419334489828067}, {'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.05, 'last_quantity': 100, 'average_price': 358.17, 'volume': 1737485, 'buy_quantity': 492304, 'sell_quantity': 757474, 'ohlc': {'open': 326.2, 'high': 368.95, 'low': 326.2, 'close': 324.65}, 'change': 11.212074541814273, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 49), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 49), 'depth': {'buy': [{'quantity': 923, 'price': 361.05, 'orders': 1}, {'quantity': 1768, 'price': 361.0, 'orders': 15}, {'quantity': 18, 'price': 360.95, 'orders': 3}, {'quantity': 232, 'price': 360.9, 'orders': 3}, {'quantity': 108, 'price': 360.85, 'orders': 2}], 'sell': [{'quantity': 237, 'price': 361.1, 'orders': 2}, {'quantity': 77, 'price': 361.4, 'orders': 1}, {'quantity': 20, 'price': 361.45, 'orders': 1}, {'quantity': 4, 'price': 361.6, 'orders': 1}, {'quantity': 53, 'price': 361.7, 'orders': 2}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 2953217, 'last_price': 2042.55, 'last_quantity': 20, 'average_price': 2047.71, 'volume': 592151, 'buy_quantity': 159593, 'sell_quantity': 360814, 'ohlc': {'open': 2059.0, 'high': 2063.75, 'low': 2032.7, 'close': 2048.3}, 'change': -0.2807205975687266, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 47), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 49), 'depth': {'buy': [{'quantity': 8, 'price': 2042.65, 'orders': 2}, {'quantity': 23, 'price': 2042.55, 'orders': 2}, {'quantity': 2, 'price': 2042.5, 'orders': 1}, {'quantity': 1, 'price': 2042.15, 'orders': 1}, {'quantity': 45, 'price': 2042.1, 'orders': 3}], 'sell': [{'quantity': 49, 'price': 2042.7, 'orders': 5}, {'quantity': 71, 'price': 2042.75, 'orders': 3}, {'quantity': 10, 'price': 2042.8, 'orders': 1}, {'quantity': 10, 'price': 2042.85, 'orders': 1}, {'quantity': 11, 'price': 2042.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'quote', 'instrument_token': 2714625, 'last_price': 358.4, 'last_quantity': 50, 'average_price': 360.74, 'volume': 2027746, 'buy_quantity': 378949, 'sell_quantity': 1412575, 'ohlc': {'open': 358.4, 'high': 363.8, 'low': 357.1, 'close': 356.65}, 'change': 0.49067713444553485}],
[{'tradable': True, 'mode': 'quote', 'instrument_token': 225537, 'last_price': 2762.1, 'last_quantity': 1, 'average_price': 2766.41, 'volume': 94835, 'buy_quantity': 54760, 'sell_quantity': 60955, 'ohlc': {'open': 2760.0, 'high': 2780.0, 'low': 2752.25, 'close': 2754.35}, 'change': 0.28137310073157007}, {'tradable': True, 'mode': 'quote', 'instrument_token': 779521, 'last_price': 317.5, 'last_quantity': 17, 'average_price': 317.52, 'volume': 3929083, 'buy_quantity': 1416289, 'sell_quantity': 2999949, 'ohlc': {'open': 318.25, 'high': 319.25, 'low': 315.35, 'close': 317.05}, 'change': 0.1419334489828067}, {'tradable': True, 'mode': 'quote', 'instrument_token': 3861249, 'last_price': 380.85, 'last_quantity': 51, 'average_price': 382.6, 'volume': 556178, 'buy_quantity': 293754, 'sell_quantity': 1267097, 'ohlc': {'open': 388.3, 'high': 388.3, 'low': 380.0, 'close': 387.15}, 'change': -1.6272762495156798}, {'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.1, 'last_quantity': 1, 'average_price': 358.17, 'volume': 1737835, 'buy_quantity': 491954, 'sell_quantity': 757341, 'ohlc': {'open': 326.2, 'high': 368.95, 'low': 326.2, 'close': 324.65}, 'change': 11.227475743107977, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 49), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 50), 'depth': {'buy': [{'quantity': 823, 'price': 361.05, 'orders': 1}, {'quantity': 1768, 'price': 361.0, 'orders': 15}, {'quantity': 18, 'price': 360.95, 'orders': 3}, {'quantity': 232, 'price': 360.9, 'orders': 3}, {'quantity': 108, 'price': 360.85, 'orders': 2}], 'sell': [{'quantity': 4, 'price': 361.1, 'orders': 1}, {'quantity': 77, 'price': 361.4, 'orders': 1}, {'quantity': 20, 'price': 361.45, 'orders': 1}, {'quantity': 4, 'price': 361.6, 'orders': 1}, {'quantity': 1, 'price': 361.7, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 340481, 'last_price': 2050.4, 'last_quantity': 4, 'average_price': 2053.2, 'volume': 299967, 'buy_quantity': 121886, 'sell_quantity': 398186, 'ohlc': {'open': 2068.0, 'high': 2072.5, 'low': 2043.55, 'close': 2059.2}, 'change': -0.4273504273504141, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 48), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 50), 'depth': {'buy': [{'quantity': 9, 'price': 2050.25, 'orders': 1}, {'quantity': 5, 'price': 2050.2, 'orders': 1}, {'quantity': 1, 'price': 2050.1, 'orders': 1}, {'quantity': 817, 'price': 2050.0, 'orders': 20}, {'quantity': 1, 'price': 2049.65, 'orders': 1}], 'sell': [{'quantity': 27, 'price': 2050.4, 'orders': 2}, {'quantity': 20, 'price': 2050.45, 'orders': 1}, {'quantity': 96, 'price': 2050.5, 'orders': 3}, {'quantity': 4, 'price': 2050.55, 'orders': 1}, {'quantity': 4, 'price': 2050.7, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 2953217, 'last_price': 2042.7, 'last_quantity': 2, 'average_price': 2047.71, 'volume': 592302, 'buy_quantity': 159417, 'sell_quantity': 360685, 'ohlc': {'open': 2059.0, 'high': 2063.75, 'low': 2032.7, 'close': 2048.3}, 'change': -0.27339745154519046, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 50), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 50), 'depth': {'buy': [{'quantity': 2, 'price': 2042.5, 'orders': 1}, {'quantity': 1, 'price': 2042.15, 'orders': 1}, {'quantity': 45, 'price': 2042.1, 'orders': 3}, {'quantity': 122, 'price': 2042.05, 'orders': 3}, {'quantity': 1943, 'price': 2042.0, 'orders': 6}], 'sell': [{'quantity': 47, 'price': 2042.7, 'orders': 5}, {'quantity': 121, 'price': 2042.75, 'orders': 4}, {'quantity': 10, 'price': 2042.8, 'orders': 1}, {'quantity': 10, 'price': 2042.85, 'orders': 1}, {'quantity': 11, 'price': 2042.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'quote', 'instrument_token': 2714625, 'last_price': 358.4, 'last_quantity': 50, 'average_price': 360.74, 'volume': 2027746, 'buy_quantity': 378949, 'sell_quantity': 1412584, 'ohlc': {'open': 358.4, 'high': 363.8, 'low': 357.1, 'close': 356.65}, 'change': 0.49067713444553485}],
[{'tradable': True, 'mode': 'full', 'instrument_token': 745473, 'last_price': 361.05, 'last_quantity': 100, 'average_price': 358.17, 'volume': 1737835, 'buy_quantity': 491954, 'sell_quantity': 757341, 'ohlc': {'open': 326.2, 'high': 368.95, 'low': 326.2, 'close': 324.65}, 'change': 11.212074541814273, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 49), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 51), 'depth': {'buy': [{'quantity': 823, 'price': 361.05, 'orders': 1}, {'quantity': 1768, 'price': 361.0, 'orders': 15}, {'quantity': 18, 'price': 360.95, 'orders': 3}, {'quantity': 232, 'price': 360.9, 'orders': 3}, {'quantity': 108, 'price': 360.85, 'orders': 2}], 'sell': [{'quantity': 4, 'price': 361.1, 'orders': 1}, {'quantity': 77, 'price': 361.4, 'orders': 1}, {'quantity': 20, 'price': 361.45, 'orders': 1}, {'quantity': 4, 'price': 361.6, 'orders': 1}, {'quantity': 1, 'price': 361.7, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 340481, 'last_price': 2050.35, 'last_quantity': 1, 'average_price': 2053.2, 'volume': 299967, 'buy_quantity': 121886, 'sell_quantity': 398186, 'ohlc': {'open': 2068.0, 'high': 2072.5, 'low': 2043.55, 'close': 2059.2}, 'change': -0.4297785547785504, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 48), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 51), 'depth': {'buy': [{'quantity': 9, 'price': 2050.25, 'orders': 1}, {'quantity': 5, 'price': 2050.2, 'orders': 1}, {'quantity': 1, 'price': 2050.1, 'orders': 1}, {'quantity': 817, 'price': 2050.0, 'orders': 20}, {'quantity': 1, 'price': 2049.65, 'orders': 1}], 'sell': [{'quantity': 27, 'price': 2050.4, 'orders': 2}, {'quantity': 20, 'price': 2050.45, 'orders': 1}, {'quantity': 96, 'price': 2050.5, 'orders': 3}, {'quantity': 4, 'price': 2050.55, 'orders': 1}, {'quantity': 4, 'price': 2050.7, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 2953217, 'last_price': 2042.05, 'last_quantity': 1, 'average_price': 2047.71, 'volume': 592302, 'buy_quantity': 159417, 'sell_quantity': 360685, 'ohlc': {'open': 2059.0, 'high': 2063.75, 'low': 2032.7, 'close': 2048.3}, 'change': -0.3051310843138323, 'last_trade_time': datetime.datetime(2019, 4, 8, 10, 28, 50), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'timestamp': datetime.datetime(2019, 4, 8, 10, 28, 51), 'depth': {'buy': [{'quantity': 2, 'price': 2042.5, 'orders': 1}, {'quantity': 1, 'price': 2042.15, 'orders': 1}, {'quantity': 45, 'price': 2042.1, 'orders': 3}, {'quantity': 122, 'price': 2042.05, 'orders': 3}, {'quantity': 1943, 'price': 2042.0, 'orders': 6}], 'sell': [{'quantity': 47, 'price': 2042.7, 'orders': 5}, {'quantity': 121, 'price': 2042.75, 'orders': 4}, {'quantity': 10, 'price': 2042.8, 'orders': 1}, {'quantity': 10, 'price': 2042.85, 'orders': 1}, {'quantity': 11, 'price': 2042.9, 'orders': 1}]}}, {'tradable': True, 'mode': 'quote', 'instrument_token': 779521, 'last_price': 317.6, 'last_quantity': 25, 'average_price': 317.52, 'volume': 3929083, 'buy_quantity': 1416289, 'sell_quantity': 2999949, 'ohlc': {'open': 318.25, 'high': 319.25, 'low': 315.35, 'close': 317.05}, 'change': 0.17347421542343838}]]


In [288]:
EQSYMBOL = lambda x:instruments_df[instruments_df['instrument_token']==x].index[0]
EQTOKEN = lambda x:instruments_df.loc[x,'instrument_token']

EQSYMBOL(225537)

EQTOKEN('DRREDDY')
LiveStream = pd.DataFrame()

In [282]:
timeStamp = dt.datetime.now().replace(second=0, microsecond=0)

def pushTick(tick):
    global LiveStream
    tick_df = pd.DataFrame(tick)
    tick_df.loc[tick_df['timestamp'].isna(), 'timestamp'] = timeStamp
    tick_df = tick_df[['timestamp','instrument_token','last_price','volume']]
    tick_df.instrument_token = tick_df.instrument_token.apply(EQSYMBOL)
    tick_df.columns = ['date','symbol','price','volume']
    tick_df.set_index(['symbol','date'], inplace=True)
    

    LiveStream = LiveStream.append(tick_df)
    LiveStream.loc['RIIL']
    LiveStream = LiveStream.groupby(['symbol','date']).agg({'price':['first','max','min','last'], 'volume':['last']})
    LiveStream.columns = LiveStream.columns.droplevel()
    LiveStream.columns = ['open', 'high','low','close', 'volume']

#pushTick(sample_tick)

In [284]:
LiveStream

,,open,high,low,close,volume
symbol,date,,,,,
ADANIPORTS,2019-04-08 15:52:00,380.90,380.90,380.90,380.90,555807
BHARTIARTL,2019-04-08 15:52:00,358.40,358.40,358.40,358.40,2027746
DRREDDY,2019-04-08 15:52:00,2762.10,2762.10,2762.10,2762.10,94835
HDFC,2019-04-08 10:28:45,2050.35,2050.35,2050.35,2050.35,299942
RIIL,2019-04-08 10:28:43,361.10,361.10,361.10,361.10,1737294
SBIN,2019-04-08 15:52:00,317.65,317.65,317.65,317.65,3928653
TCS,2019-04-08 10:28:45,2042.65,2042.65,2042.65,2042.65,592121


In [283]:
for tick in sample_ticks:
    pushTick(sample_tick)

In [293]:
#f2 = open('live_data_hdfc_full_4_8_2019.txt', 'rb')
unpickled_df = pd.read_pickle('live_data_hdfc_full_4_8_2019.txt')

In [297]:
unpickled_df

[{'tradable': True,
  'mode': 'quote',
  'instrument_token': 225537,
  'last_price': 2762.1,
  'last_quantity': 1,
  'average_price': 2766.41,
  'volume': 94835,
  'buy_quantity': 54745,
  'sell_quantity': 60966,
  'ohlc': {'open': 2760.0, 'high': 2780.0, 'low': 2752.25, 'close': 2754.35},
  'change': 0.28137310073157007},
 {'tradable': True,
  'mode': 'quote',
  'instrument_token': 779521,
  'last_price': 317.65,
  'last_quantity': 135,
  'average_price': 317.52,
  'volume': 3928653,
  'buy_quantity': 1407649,
  'sell_quantity': 3003899,
  'ohlc': {'open': 318.25, 'high': 319.25, 'low': 315.35, 'close': 317.05},
  'change': 0.18924459864373627},
 {'tradable': True,
  'mode': 'quote',
  'instrument_token': 3861249,
  'last_price': 380.9,
  'last_quantity': 43,
  'average_price': 382.6,
  'volume': 555807,
  'buy_quantity': 293804,
  'sell_quantity': 1266317,
  'ohlc': {'open': 388.3, 'high': 388.3, 'low': 380.0, 'close': 387.15},
  'change': -1.6143613586465195},
 {'tradable': True,
  

# Visualization

In [79]:
plotData()